# Configuration
First, we specify the SLURM server and login information:

In [1]:
import dashboard
davinci = dashboard.SLURMServer('davinci.rice.edu', 'jmd11', 'C:/Users/Jeffrey/.ssh/id_rsa')

Next, we choose a local directory on the current machine and a remote directory on the SLURM cluster. Note these directories will be created (when a job is started) if they do not currently exist.

In [2]:
from slurm_experiment import ExperimentConfig
config = ExperimentConfig(server=davinci,
                          local_directory='C:/Work/Projects/SlurmQueen/example',
                          remote_directory='/scratch/jmd11/experiments/slurmqueen')

# Defining an experiment

When running many experiments on the same tool, it is convenient to define a subclass of SlurmExperiment to hold any additional server setup.

In [3]:
from slurm_experiment import SlurmExperiment

class SlurmTest(SlurmExperiment):
    def __init__(self, experiment_id, changing_args):
        SlurmExperiment.__init__(self, 'experiments/' + experiment_id, 'python example_tool.py', changing_args,
                                 ['example_tool.py'],
                                 setup_commands='\n'.join(["module load Anaconda3/5.0.0"]))

We can then define a single experiment on ```example_tool.py``` by providing a name for this experiment and a list of tasks. Each task is defined by a set of arguments, given by a dictionary of key/value pairs. These arguments are passed as arguments to the tools as ```--key=value``` arguments, with a few exceptions:
* The key ```''```, if it exists, indicates a list of positional arguments (which are given in the provided order). 
* Keys that begin with ```'|'``` are not passed to each task. Such keys can be used when processing results.

In [4]:
slurm_test_1 = SlurmTest('slurm_test_1',
                         [{'': [chr(a+65) + chr(b+65)],
                           'a': a, 'b': b,
                           '|desc': '%d + %d' % (a, b)
                          } for a in range(3) for b in range(3)])

## Running an experiment

Once we have defined an experiment and a configuration, we can run the experiment on the provided cluster in a single command. In this case, we run the 9 tasks using 2 SLURM workers. Each worker is given a timeout of 5 minutes. The following command performs each of the following steps:
1. Creates a set of 9 .in files in ```experiments/slurm_test_1/```, each defining a single task.
2. Copies all .in files and all dependencies provided to the ```SlurmExperiment``` to the SLURM cluster, in the folder indicated in ```config```
3. Generates an appropriate SLURM script to run all tasks on the provided number of workers (distributed in a round-robin way).
4. Submits the SLURM job, returning the job id.

In [5]:
slurm_test_1.slurm_instance(config).run(2, '5:00')

Connected to davinci.rice.edu
Created 9 local files
Compressed local files
Copied files to remote server
Attempting to submit job
Submitted batch job 4904649



Once an experiment has finished running, we use a single additional command to download all results back to the local machine (and clean up the files on the cluster).

In [6]:
slurm_test_1.slurm_instance(config).complete()

Experiment complete. Compressing and copying results.
Deleting files from remote server: /scratch/jmd11/experiments/slurmqueen/experiments/slurm_test_1/


We can then use an SQL interface to query the results.

In [7]:
slurm_test_1.slurm_instance(config).query('SELECT * FROM data')

Reading all output data into SQL table


,Repeated Text,Sum,a,b,file,output,|desc
0,AAAAAA,0,0,0,0,./0.out,0 + 0
1,ABABAB,1,0,1,1,./1.out,0 + 1
2,ACACAC,2,0,2,2,./2.out,0 + 2
3,BABABA,1,1,0,3,./3.out,1 + 0
4,BBBBBB,2,1,1,4,./4.out,1 + 1
5,BCBCBC,3,1,2,5,./5.out,1 + 2
6,CACACA,2,2,0,6,./6.out,2 + 0
7,CBCBCB,3,2,1,7,./7.out,2 + 1
8,CCCCCC,4,2,2,8,./8.out,2 + 2


## (Optional) Generating and analyzing an experiment without a SLURM cluster.

To aid in reproducability, an experiment can be generated and analyzed even without access to a SLURM cluster. In particular, the code in ```experiment.py``` is sufficient to generate all ```*.in``` files and provide an SQL interface to results.

We can define an experiment here purely as the command and set of arguments.

In [8]:
from experiment import Experiment

experiment_1 = Experiment('python example_tool.py',
                          [{'': [chr(a+65) + chr(b+65)],
                            'a': a, 'b': b,
                            '|desc': '%d + %d' % (a, b)
                           } for a in range(3) for b in range(3)])

We then only need to provide a local directory in order to generate all ```*.in``` files.

In [9]:
experiment_1.instance('experiments/slurm_test_1').setup()

Each ```*.in``` file is a complete bash script that runs a single task and produces the corresponding ```.out``` files. Once these ```.out``` files are computated separately or previously provided, the SQL interface can still be used to find the results.

In [10]:
experiment_1.instance('experiments/slurm_test_1').query('SELECT * FROM data')

,Repeated Text,Sum,a,b,file,output,|desc
0,AAAAAA,0,0,0,0,./0.out,0 + 0
1,ABABAB,1,0,1,1,./1.out,0 + 1
2,ACACAC,2,0,2,2,./2.out,0 + 2
3,BABABA,1,1,0,3,./3.out,1 + 0
4,BBBBBB,2,1,1,4,./4.out,1 + 1
5,BCBCBC,3,1,2,5,./5.out,1 + 2
6,CACACA,2,2,0,6,./6.out,2 + 0
7,CBCBCB,3,2,1,7,./7.out,2 + 1
8,CCCCCC,4,2,2,8,./8.out,2 + 2
